# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [ ]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [ ]:
# Buscar cidades do Vale do Paraíba
cities = {}
IBGE_URL = "https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios"

response = requests.get("{}".format(IBGE_URL))
json_response = response.json()
for city in json_response:
      city['nome'] = unidecode.unidecode(city['nome'])

# Criar data frame com as cidades
cities_raw = spark.sparkContext.parallelize([json_response])
cities = spark.read.json(cities_raw)

# Criar view com as cidades
cities.createOrReplaceTempView("cities")

In [ ]:
# Buscar previsão do tempo para as cidades
M3O_API_TOKEN="ZTY4NmQ0MjctNTViOC00MjhjLTk5ZjMtYWQwZjE1Y2IwOGY3"
SEARCH_CITY = "https://api.m3o.com/v1/weather/Forecast"

session = requests.Session()
session.headers = {"Content-Type":"application/json","Authorization": f"Bearer {M3O_API_TOKEN}"}
cidades =  []
for cidade in cities.collect():
      while True:            
            response = session.post(SEARCH_CITY,json={"days": 5, "location" : cidade[2]})
            if response.status_code == 200:
                  probabily_cities = response.json() 
                  for x in probabily_cities['forecast']:
                        if x['will_it_rain'] == True:
                              x['will_it_rain'] = 'Sim'
                        else:
                              x['will_it_rain'] = 'Não'
                  cidades.append(probabily_cities)
                  break
            elif (response.status_code == 500):
                  break # cidade não existe no M3o
            else:
                  continue

# Criar data frame com as previsões
forecast_raw = spark.sparkContext.parallelize(cidades)
forecast = spark.read.json(forecast_raw)
forecast = forecast.filter(forecast['region'] == "Sao Paulo")

# Criar view com as previsões
forecast.createOrReplaceTempView("forecast")

In [ ]:
# Criar DF da Tabela 1
SQL_TABLE_1 = "SELECT f.location AS Cidade,cities.id AS CodigoDaCidade,f.date AS Data,f.region AS Regiao,'Brasil' AS Pais,f.latitude AS Latitude,f.longitude AS Longigute,f.max_temp_c AS TemperaturaMaxima,f.min_temp_c AS TemperaturaMinima,f.avg_temp_c AS TemperaturaMedia,f.will_it_rain AS VaiChover,CONCAT(f.chance_of_rain,'%') AS ChanceDeChuva,f.condition AS CondicaoDoTempo,f.sunrise AS NascerDoSol,f.sunset AS PorDoSol,f.max_wind_kph AS VelocidadeMaximaDoVento FROM (SELECT f.location,f.local_time,f.region,f.latitude,f.longitude,inline(f.forecast) FROM forecast AS f) AS f INNER JOIN cities on cities.nome = f.location"
tabela_1 = spark.sql(SQL_TABLE_1)# Longigute é o solicitado no notebook

In [ ]:
# Criar DF da Tabela 2
SQL_TABLE_2 = "SELECT f.location AS Cidade, SUM(if(f.will_it_rain = 'Sim', 1, 0)) AS QtdDiasVaiChover, SUM(if(f.will_it_rain = 'Não', 1, 0)) AS QtdDiasNaoVaiChover, COUNT(f.location) AS TotalDiasMapeados FROM (SELECT f.location,inline(f.forecast) FROM forecast AS f) AS f GROUP BY f.location"
tabela_2 = spark.sql(SQL_TABLE_2)

In [ ]:
# Exportar CSVs
tabela_1.coalesce(1).write.csv("/tmp/spark_output/tabela1", mode="append", header="true",sep=';',encoding='ISO-8859-1')#coalesce(1) para deixar em um unico csv final
tabela_2.coalesce(1).write.csv("/tmp/spark_output/tabela2", mode="append", header="true",sep=';',encoding='ISO-8859-1')